# 数据源

## 文件

文件是最常见的数据来源之一，例如Excel文件、csv文件等。  

In [12]:
# 利用`pandas.read_csv`方法来读取一个csv文件

import pandas as pd

df = pd.read_csv('data/nba_players.csv')
df.head()

,player_id,player_name,avg_pts,avg_ast,avg_oreb,avg_dreb,avg_stl,avg_blk,avg_tov,avg_fgm,avg_fga,avg_tpm,avg_play_time,team_name,position
0,1,凯尔-科沃尔,10.12,1.64,0.13,2.66,0.49,0.31,1.03,3.57,7.66,2.42,26.13,骑士,2.5
1,2,蒂亚戈-斯普利特,4.33,0.44,0.89,1.56,0.11,0.11,0.67,1.56,3.44,0.22,8.56,76人,5.0
2,3,保罗-米尔萨普,17.87,3.63,1.59,6.13,1.30,0.87,2.29,6.17,13.94,1.09,33.66,老鹰,4.0
3,4,萨博-塞福洛沙,7.16,1.73,0.84,3.52,1.48,0.50,0.95,2.81,6.35,0.66,25.73,老鹰,2.5
4,5,杰夫-蒂格,15.29,7.78,0.39,3.66,1.24,0.39,2.63,4.90,11.07,1.10,32.44,步行者,1.0


`pandas`提供了读取多种常用文件类型的方法，例如常用的有
- `pandas.read_csv`读取csv文件，可以通过设置`sep`可以读取不同分隔符的文件，例如`tab`为分割符的文件
- `pandas.read_excel`读取Excel文件（注意需要安装`openpyxl`库）
- `pandas.read_json`读取JSON文件
- `pandas.read_sas`读取SAS文件
- `pandas.read_ssps`读取SPSS文件

此外，`pandas`还提供了一个`read_clipboard`方法，可以方便读取你`复制`中的内容，例如我们复制一下内容:  

>
date,h,l,w  
2020-01-01,19,10,rainy  
2020-01-02,18,11,sunny  
2020-01-03,17,10,sunny  
>

In [3]:
# 读取复制中的内容

pd.read_clipboard(sep=',')

,date,h,l,w
0,2020-01-01,19,10,rainy
1,2020-01-02,18,11,sunny
2,2020-01-03,17,10,sunny


这样，我们可以方便的复制Excel中的数据，然后使用`read_clipboard`方法，可以快速读取数据  
例如  
![excel_screenshot](resource/chp1_excel_clipboard.png)

In [6]:
# 读取复制的Excel表中的内容

pd.read_clipboard(sep='\t')

,id,name,score
0,1,Marry,8.2
1,2,Peter,9.1
2,3,Jhon,8.6


## 数据库

在大多数工程环境下，数据库是数据最主要的来源，大部分数据都来源于数据库。`pandas`中提供了`read_sql`方法，可以通过sql语句来读取数据库中的数据，这是我们最常用的读取数据库中数据的方法。  
`read_sql`需要设置两个必要参数，`sql`和`con`

- `sql`：sql语句  
- `con`：数据库的链接  
    创建获取数据库链接，我们可以使用数据库相应的库来完成，例如`sqlite3`、`pymysql`等  

例如，我们从本地的`MySQL`数据库读取`user`表中的数据

~~~Python
import pymysql
import pandas as pd

conn = pymysql.connect(
    host='127.0.0.1',   # MySQL地址
    port=3306,          # MySQL端口
    user='root',        # MySQL用户名
    password='123456',  # MySQL密码
    database='test'     # MySQL数据库
)
sql = 'select * from user'
data = pd.read_sql(sql, conn)
# 关闭MySQL链接
conn.close() 
~~~

此外还有一个比较常用的参数是`parse_dates`，设置这个参数可以将指定的某几列转化为日期类型。

In [10]:
# 我们读取SQLite库中的user表中的全部数据

import sqlite3

conn = sqlite3.connect('data/sqlite_db.db')
df = pd.read_sql('SELECT * FROM user', conn)
conn.close()

df.head()

,id,name,score,level
0,1,Mary,9.3,A
1,2,Peter,8.1,B
2,3,Jhon,8.2,B
3,4,Jane,7.4,C


## API

In [18]:
# 从API获取数据

import json
import requests

response = requests.get('http://wthrcdn.etouch.cn/weather_mini?city=上海')
content = json.loads(response.content)
content

{'data': {'aqi': '48',
  'city': '上海',
  'forecast': [{'date': '9日星期四',
    'fengli': '<![CDATA[<3级]]>',
    'fengxiang': '北风',
    'high': '高温 7℃',
    'low': '低温 6℃',
    'type': '小雨'},
   {'date': '10日星期五',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '东北风',
    'high': '高温 10℃',
    'low': '低温 8℃',
    'type': '小雨'},
   {'date': '11日星期六',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '东北风',
    'high': '高温 9℃',
    'low': '低温 5℃',
    'type': '小雨'},
   {'date': '12日星期天',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '北风',
    'high': '高温 9℃',
    'low': '低温 4℃',
    'type': '阴'},
   {'date': '13日星期一',
    'fengli': '<![CDATA[3-4级]]>',
    'fengxiang': '东风',
    'high': '高温 10℃',
    'low': '低温 6℃',
    'type': '阴'}],
  'ganmao': '天气寒冷，昼夜温差极大且空气湿度较大，易发生感冒，请注意适当增减衣服，加强自我防护避免感冒。',
  'wendu': '6',
  'yesterday': {'date': '8日星期三',
   'fl': '<![CDATA[3-4级]]>',
   'fx': '西北风',
   'high': '高温 7℃',
   'low': '低温 4℃',
   'type': '多云'}},
 'desc': 'OK',
 'status': 1000}

In [28]:
# 将从API获取的数据中forecast数据转换成DataFrame

forecast = content['data']['forecast']
df = pd.DataFrame(forecast)
df

,date,high,fengli,low,fengxiang,type
0,9日星期四,高温 7℃,<![CDATA[<3级]]>,低温 6℃,北风,小雨
1,10日星期五,高温 10℃,<![CDATA[3-4级]]>,低温 8℃,东北风,小雨
2,11日星期六,高温 9℃,<![CDATA[3-4级]]>,低温 5℃,东北风,小雨
3,12日星期天,高温 9℃,<![CDATA[3-4级]]>,低温 4℃,北风,阴
4,13日星期一,高温 10℃,<![CDATA[3-4级]]>,低温 6℃,东风,阴
